# Optimización de f(x,y) = -tan²(0.5(sin x + sin y))

Este notebook permite ejecutar **individualmente cada método de optimización**:

- Newton con paso fijo
- Newton amortiguado (damped) con búsqueda lineal
- BFGS

Al ejecutar la celda correspondiente a un método, se muestran:

1. Convergencia de f(x,y) vs iteración
2. Trayectorias sobre contornos de la función
3. Valor final y número de iteraciones

Puedes cambiar el punto inicial dentro de cada celda para probar distintas condiciones.


In [52]:
import numpy as np
import matplotlib.pyplot as plt

# === Definición de la función objetivo ===
def f(x):
    u = 0.5*(np.sin(x[0]) + np.sin(x[1]))
    return -np.tan(u)**2

# === Gradiente ===
def grad_f(x):
    u = 0.5*(np.sin(x[0]) + np.sin(x[1]))
    sec2 = 1 / np.cos(u)**2
    du_dx = 0.5*np.cos(x[0])
    du_dy = 0.5*np.cos(x[1])
    return -2*np.tan(u)*sec2*np.array([du_dx, du_dy])

# === Hessiana ===
def hess_f(x):
    u = 0.5*(np.sin(x[0]) + np.sin(x[1]))
    sec2 = 1/np.cos(u)**2
    tan_u = np.tan(u)
    du_dx = 0.5*np.cos(x[0])
    du_dy = 0.5*np.cos(x[1])
    d2u_dx2 = -0.5*np.sin(x[0])
    d2u_dy2 = -0.5*np.sin(x[1])
    H = np.zeros((2,2))
    d2f_du2 = -2*(sec2**2)*(1 + 2*tan_u**2)
    df_du = -2*tan_u*sec2
    for i, (du_i, d2u_i2) in enumerate(zip([du_dx, du_dy], [d2u_dx2, d2u_dy2])):
        H[i,i] = d2f_du2*du_i**2 + df_du*d2u_i2
    H[0,1] = H[1,0] = d2f_du2*du_dx*du_dy
    return H


In [140]:
def newton_fixed_robust(x0, alpha=1.0, tol=1e-6, max_iter=100):
    x = np.array(x0, dtype=float)
    g = grad_f(x)
    tray = []

    # Convergencia inicial
    if np.linalg.norm(g) < tol:
        tray.append(x.copy())
        print(f"\nMétodo: Newton (paso fijo α={alpha}) — CONVERGIÓ")
        print(f"Punto final: {x}")
        print(f"f* = {f(x):.6f}")
        print(f"Iteraciones: 0")
        return

    tray.append(x.copy())
    diverged = False

    for k in range(max_iter):
        try:
            H = hess_f(x)
            H += 1e-6*np.eye(2)
            p = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            print(f"⚠️ Hessiana singular en iter {k}")
            diverged = True
            break

        # Limitar tamaño del paso
        if np.linalg.norm(p) > 1.0:
            p = p / np.linalg.norm(p)

        x = x + alpha*p
        tray.append(x.copy())
        g = grad_f(x)

        if np.linalg.norm(g) < tol:
            break

        fx_new = f(x)
        if np.isnan(fx_new) or np.isinf(fx_new) or np.linalg.norm(x) > 1e6:
            diverged = True
            break

    estado = "DIVERGIÓ" if diverged else "CONVERGIÓ"
    print(f"\nMétodo: Newton (paso fijo α={alpha}) — {estado}")
    print(f"Punto final: {x}")
    print(f"f* = {f(x):.6f}")
    print(f"Iteraciones: {len(tray)-1}")  # restamos el punto inicial

    
newton_fixed_robust([99,99])    



Método: Newton (paso fijo α=1.0) — CONVERGIÓ
Punto final: [98.9601686 98.9601686]
f* = -2.425519
Iteraciones: 2


In [139]:
def newton_damped_robust(x0, tol=1e-6, max_iter=100):
    x = np.array(x0, dtype=float)
    g = grad_f(x)
    tray = []

    if np.linalg.norm(g) < tol:
        tray.append(x.copy())
        print(f"\nMétodo: Newton Damped — CONVERGIÓ")
        print(f"Punto final: {x}")
        print(f"f* = {f(x):.6f}")
        print(f"Iteraciones: 0")
        return

    tray.append(x.copy())
    diverged = False

    for k in range(max_iter):
        try:
            H = hess_f(x)
            H += 1e-6*np.eye(2)
            p = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            print(f"⚠️ Hessiana singular en iter {k}")
            diverged = True
            break

        if np.linalg.norm(p) > 1.0:
            p = p / np.linalg.norm(p)

        # Backtracking line search
        alpha = 1.0
        while f(x + alpha*p) > f(x) + 1e-4*alpha*np.dot(g, p):
            alpha *= 0.5
            if alpha < 1e-6:
                diverged = True
                break

        x = x + alpha*p
        tray.append(x.copy())
        g = grad_f(x)

        if np.linalg.norm(g) < tol:
            break

        fx_new = f(x)
        if np.isnan(fx_new) or np.isinf(fx_new) or np.linalg.norm(x) > 1e6:
            diverged = True
            break

    estado = "DIVERGIÓ" if diverged else "CONVERGIÓ"
    print(f"\nMétodo: Newton Damped — {estado}")
    print(f"Punto final: {x}")
    print(f"f* = {f(x):.6f}")
    print(f"Iteraciones: {len(tray)-1}")

    
newton_damped_robust([0,np.pi])    



Método: Newton Damped — CONVERGIÓ
Punto final: [0.         3.14159265]
f* = -0.000000
Iteraciones: 0


In [135]:
def bfgs_robust(x0, tol=1e-6, max_iter=100):
    x = np.array(x0, dtype=float)
    g = grad_f(x)
    tray = []

    if np.linalg.norm(g) < tol:
        tray.append(x.copy())
        print(f"\nMétodo: BFGS — CONVERGIÓ")
        print(f"Punto final: {x}")
        print(f"f* = {f(x):.6f}")
        print(f"Iteraciones: 0")
        return

    tray.append(x.copy())
    B = np.eye(2)
    diverged = False

    for k in range(max_iter):
        p = -np.linalg.solve(B, g)

        if np.linalg.norm(p) > 1.0:
            p = p / np.linalg.norm(p)

        # Backtracking line search
        alpha = 1.0
        while f(x + alpha*p) > f(x) + 1e-4*alpha*np.dot(g, p):
            alpha *= 0.5
            if alpha < 1e-6:
                diverged = True
                break

        s = alpha*p
        x_new = x + s
        fx_new = f(x_new)

        if np.isnan(fx_new) or np.isinf(fx_new) or np.linalg.norm(x_new) > 1e6:
            diverged = True
            break

        y = grad_f(x_new) - g
        if np.dot(y, s) > 1e-10:
            Bs = B @ s
            B = B + np.outer(y, y)/np.dot(y, s) - np.outer(Bs, Bs)/np.dot(s, Bs)

        x = x_new
        g = grad_f(x)
        tray.append(x.copy())

        if np.linalg.norm(g) < tol:
            break

    estado = "DIVERGIÓ" if diverged else "CONVERGIÓ"
    print(f"\nMétodo: BFGS — {estado}")
    print(f"Punto final: {x}")
    print(f"f* = {f(x):.6f}")
    print(f"Iteraciones: {len(tray)-1}")

    
bfgs_robust([0.1,2])    



Método: BFGS — CONVERGIÓ
Punto final: [1.57079634 1.57079635]
f* = -2.425519
Iteraciones: 8
